In [1]:
import io
import os
import requests
import pandas as pd
import openpyxl 

In [2]:
file_path = '../data/penn_world_table.xlsx'

# Load workbook and get sheet names
workbook = openpyxl.load_workbook(file_path)
sheet_names = workbook.sheetnames

# Display sheet names for reference
print("Sheet names in the Excel file:")
for i, sheet_name in enumerate(sheet_names):
    print(f"{i}: {sheet_name}")

# Read all sheets into DataFrames
dfs = [pd.read_excel(file_path, engine="openpyxl", sheet_name=sheet_name) for sheet_name in sheet_names]

# ✅ Find the first DataFrame with the required columns
common_columns = ['countrycode', 'year']
df_combined = None

for df in dfs:
    df.columns = df.columns.str.strip()  # Strip whitespace from column names
    if {'countrycode', 'year'}.issubset(df.columns):
        df_combined = df
        break

# ✅ If no DataFrame has the required columns, raise an error
if df_combined is None:
    raise ValueError("None of the sheets contain the required columns: 'countrycode' and 'year'.")

# ✅ Merge the remaining DataFrames if they have the required columns
for i, df in enumerate(dfs):
    if df is not df_combined and {'countrycode', 'year'}.issubset(df.columns):
        df_combined = pd.merge(df_combined, df, on=common_columns, how='left', 
                               suffixes=(f'_{i-1}', f'_{i}'))

# ✅ Basic data cleaning
df_combined = df_combined.dropna()  # Drop rows with missing values
df_combined.columns = df_combined.columns.str.strip()  # Strip whitespace from column names
df_combined = df_combined.sort_values(by=['countrycode', 'year']).reset_index(drop=True)  # Sort and reset index

# ✅ Display final DataFrame
print("\nCombined and Cleaned DataFrame:")
print(df_combined.head())

Sheet names in the Excel file:
0: Info
1: Legend
2: rgdpna
3: rnna
4: emp
5: hc
6: pop
7: labsh

Combined and Cleaned DataFrame:
  countrycode country_2 currency_unit_2  year        rgdpna country_3  \
0         AGO    Angola          Kwanza  1970  54237.054688    Angola   
1         AGO    Angola          Kwanza  1971  57491.277344    Angola   
2         AGO    Angola          Kwanza  1972  57606.261719    Angola   
3         AGO    Angola          Kwanza  1973  62272.367188    Angola   
4         AGO    Angola          Kwanza  1974  64202.808594    Angola   

  currency_unit_3          rnna country_4 currency_unit_4       emp country_5  \
0          Kwanza  295517.62500    Angola          Kwanza  3.666207    Angola   
1          Kwanza  314195.09375    Angola          Kwanza  3.742484    Angola   
2          Kwanza  332435.84375    Angola          Kwanza  3.853271    Angola   
3          Kwanza  352647.90625    Angola          Kwanza  3.987807    Angola   
4          Kwanza  373267.7

In [4]:
# Save DataFrame as Pickle file
df_combined.to_pickle('../data/df_combined.pkl')
